#### Script to check rf and compute the visuomemory index block 1
- First, check if neuron responding to the task by comparing activity during trial vs baseline
- Second, check whether the rf is in in or out
- Third, compute the visuomemory index

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import signal,stats
import pandas as pd
from typing import Dict
from collections import defaultdict
from ephysvibe.trials.spikes import firing_rate,sp_constants,plot_raster
from ephysvibe.trials import select_trials
from ephysvibe.spike_sorting import config
from ephysvibe.structures.bhv_data import BhvData
from ephysvibe.structures.spike_data import SpikeData
from ephysvibe.analysis import raw_ch,circular_stats
from ephysvibe.task import task_constants,def_task
import os 
seed=2023

### Def functions

In [144]:
def get_neurons_info(sp_samples:np.ndarray,fix_t:int,neuron_type:np.ndarray,task:pd.DataFrame,dur_v:int,st_m:int,end_m:int,st_t:int,end_t:int,min_trials:int=5,n_spikes_sec:int=5)->pd.DataFrame:
    trial_dur = sp_samples.shape[2]
    neurons_info:Dict[str,list] = defaultdict(list)
    i_good,i_mua, n_type = 0,0,0
    for i_neuron,type_neuron in enumerate(neuron_type):
        
        # check and count type of unit
        if type_neuron == 'good':
            i_good+=1
            n_type = i_good
        elif type_neuron == 'mua':
            i_mua+=1
            n_type = i_mua
        max_in_out = np.array([0,0])
        larger,v_larger,p_larger = np.array([False,False]),np.array([False,False]),np.array([False,False])
        p_in_out,p_v,p_p = np.array([None,None]),np.array([None,None]),np.array([None,None])
        for i,in_out in enumerate(["in","out"]): # iterate by code'
            trial_idx = task[(task['i_neuron']==i_neuron)&(task['in_out']==in_out)]["trial_idx"]
            trial_idx = trial_idx[(sp_samples[trial_idx,i_neuron,fix_t:].sum(axis=1)>0)]
            n_tr = len(trial_idx)
            true_in_out = 'in'
            if n_tr > 3:
                mean_visual = sp_samples[trial_idx,i_neuron,fix_t+50:fix_t+dur_v].mean(axis=1)
                mean_prep = sp_samples[trial_idx,i_neuron,fix_t+st_m:fix_t+end_m].mean(axis=1)
                mean_test = sp_samples[trial_idx,i_neuron,fix_t+st_t:fix_t+end_t].mean(axis=1)
                mean_bl = sp_samples[trial_idx,i_neuron,:fix_t].mean(axis=1)
                v_larger[i] = mean_bl.mean()<mean_visual.mean()
                p_larger[i] = mean_bl.mean()<mean_prep.mean()
                t_larger = mean_bl.mean()<mean_test.mean()
                larger[i]= v_larger[i] or p_larger[i] or t_larger
                p_v[i] = stats.ttest_rel(mean_bl, mean_visual)[1]
                p_p[i] = stats.ttest_rel(mean_bl, mean_prep)[1]
                p_t = stats.ttest_rel(mean_bl, mean_test)[1]

                p_in_out[i] = np.min([p_v[i],p_p[i],p_t])
                max_in_out[i] = np.max([v_larger[i],p_larger[i],t_larger])
        for i,in_out in enumerate(["in","out"]): # iterate by code'
            if np.argmax(max_in_out) == 1 or p_in_out[0]>0.05:
                true_in_out='out'
            neurons_info['array_position']+=[i_neuron]
            neurons_info['cluster']+=[n_type]
            neurons_info['group']+=[type_neuron]            
            neurons_info['in_out']+=[in_out]
            neurons_info['true_in_out']+=[true_in_out]
            neurons_info['p']+=[p_in_out[1]]
            neurons_info['larger']+=[larger[i]]  
            neurons_info['v_larger']+=[v_larger[i]]  
            neurons_info['p_v']+=[p_v[i]]  
            neurons_info['p_larger']+=[p_larger[i]]  
            neurons_info['p_p']+=[p_p[i]]  
            
   
    neurons_info = pd.DataFrame(neurons_info)
    return neurons_info

### Start analysis

In [145]:
# load data
filepath = "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/session_struct/lip/spikes/2023-03-17_10-11-51_Riesling_lip_e1_r1_sp.h5"  #2023-02-27_10-50-03 2023-02-09_10-42-54 2023-03-21_10-40-02
bhv_path = '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/session_struct/bhv/2023-03-17_10-11-51_Riesling_e1_r1_bhv.h5'
s_path = os.path.normpath(filepath).split(os.sep)
data = SpikeData.from_python_hdf5(filepath)
bhv = BhvData.from_python_hdf5(bhv_path)

In [149]:
# Select trials and create task frame
block = 1
trials_block = np.where(np.logical_and(data.trial_error == 0, data.block == block))[0]
print("Number of clusters: %d" % len(data.clustersgroup))

Number of clusters: 10


In [150]:
if np.any(np.isnan(data.neuron_cond)):
    neuron_cond = np.ones(len(data.clustersgroup))
else:
    neuron_cond = data.neuron_cond
task = def_task.create_task_frame(
    condition=bhv.condition[trials_block],
    test_stimuli=bhv.test_stimuli[trials_block],
    samples_cond=task_constants.SAMPLES_COND,
    neuron_cond=neuron_cond,
)

In [151]:
sp_samples = data.sp_samples
neuron_type = data.clustersgroup
code_samples = data.code_samples
code_numbers = data.code_numbers
sp_samples = data.sp_samples

fix_t = 200
dur_v = 250
st_m = 450 #800
end_m = 900
st_t = 900
end_t = 900+450
min_trials = 3
n_spikes_sec = 5
p_threshold = 0.05
vm_threshold = 0.4

In [152]:
shifts_on = code_samples[:,3]  
shifts_on = shifts_on[:,np.newaxis]
shifts_test = code_samples[:,6]
shifts_test = shifts_test[:,np.newaxis]
shifts_test = np.where(np.isnan(shifts_test),0,shifts_test)
sp_shift_on = SpikeData.indep_roll(sp_samples, -(shifts_on-fix_t).astype(int), axis=2)[:,:,:end_m+fix_t]
sp_shift_test = SpikeData.indep_roll(sp_samples, -(shifts_test).astype(int), axis=2)[:,:,:450]
sp_shift = np.concatenate([sp_shift_on,sp_shift_test],axis=2)

#### Select responding neurons and check rf

In [153]:
neurons_info = get_neurons_info(sp_shift,fix_t,neuron_type,task,dur_v=dur_v,st_m=st_m,end_m=end_m,st_t=st_t,end_t=end_t,min_trials=min_trials,n_spikes_sec=n_spikes_sec)
neurons_info = neurons_info[(neurons_info['p']<0.05)& (neurons_info['larger']==True)] # responding neurons


#### Compute vm index

In [154]:
neurons_info = neurons_info[neurons_info["in_out"]==neurons_info["true_in_out"]]
neurons_info = neurons_info[(np.logical_and(neurons_info["p_v"]<0.05,neurons_info["v_larger"]==True)) | (np.logical_and(neurons_info["p_p"]<0.05,neurons_info["p_larger"]==True))]

In [142]:
neurons_info

,array_position,cluster,group,in_out,true_in_out,p,larger,v_larger,p_v,p_larger,p_p
6,3,4,good,in,in,4.186223e-06,True,False,3.956539e-05,True,0.003817
8,4,1,mua,in,in,1.534534e-05,True,False,6.402042e-01,True,0.000669
12,6,2,mua,in,in,7.456515e-06,True,True,2.163197e-01,True,0.000122
14,7,3,mua,in,in,1.197297e-09,True,True,1.021010e-08,True,0.000010
16,8,6,good,in,in,5.780027e-03,True,True,2.209263e-02,True,0.125446


In [ ]:
def get_vm_index(th_rf,target_codes,sp_samples,sp_bl,align_event,fix_t,dur_v,st_m,end_m,n_spikes_sec,min_trials):
    test_vm:Dict[str,list] = defaultdict(list)
    for _, row in th_rf.iterrows():
        i_neuron = row['array_position']
        code = row['code']
        vm_index=np.nan
        sig_type=np.nan
        # select trials

        target_t_idx = np.array(target_codes[code]["trial_idx"]) # select trials with the same stimulus
        target_t_idx = target_t_idx[(sp_samples[target_t_idx,i_neuron].sum(axis=1)>0)]
        all_trials_sp = []
        for i_code in target_codes.keys():
            all_trials = np.array(target_codes[i_code]["trial_idx"]) # select trials with the same stimulus
            all_trials = all_trials[(sp_samples[all_trials,i_neuron].sum(axis=1)>0)]
            all_trials = sp_samples[all_trials,i_neuron]#.mean(axis=0)

            all_trials_sp.append([all_trials[:,50:dur_v].mean(),all_trials[:,st_m:end_m].mean()])

        all_trials_sp = np.concatenate(all_trials_sp)
        min_sp = np.nanmin(all_trials_sp)
        max_sp = np.nanmax(all_trials_sp)-min_sp

        if target_t_idx.shape[0]<=min_trials:
            next_idx,prev_idx = get_trials(code,target_codes)
            n_tr_min = np.min([len(next_idx),len(prev_idx)])
            rng = np.random.default_rng(seed=seed)
            next_idx=rng.choice(next_idx, size=n_tr_min, replace=False)
            prev_idx=rng.choice(prev_idx, size=n_tr_min, replace=False)
            target_t_idx = np.concatenate([target_t_idx,next_idx,prev_idx])  
            target_t_idx = target_t_idx[(sp_samples[target_t_idx,i_neuron].sum(axis=1)>0)]
        if target_t_idx.shape[0]>=min_trials:
            sp_code = sp_samples[target_t_idx,i_neuron]# Select trials with at least  5 spikes/sec
            sp_bl_code = sp_bl[target_t_idx,i_neuron,:200]

            sp_trial_avg = sp_code.mean(axis=0)
            v_mean=((sp_trial_avg[ 50:dur_v].mean()-min_sp)/max_sp)
            m_mean=((sp_trial_avg[ st_m:end_m].mean()-min_sp)/max_sp)
            vm_index = (m_mean-v_mean)/(v_mean+m_mean)
            if vm_index <=0: sig_type = "visual"
            else: sig_type="anticipation" 
        # save results
        test_vm['code']+=[code]
        test_vm['array_position']+=[i_neuron]
        test_vm['vm_index']+=[vm_index]
        test_vm['sig_type']+=[sig_type]  
        test_vm['cluster']+=[row['cluster']]
        test_vm['group']+=[row['group']]
        test_vm['p_v']+=[row['p_v'] ]  
        test_vm['v_larger']+=[row['v_larger']] 
        test_vm['p_p']+=[row['p_p'] ]  
        test_vm['p_larger']+=[row['p_larger']] 
    test_vm = pd.DataFrame(test_vm)
    return test_vm